

# CNN Implementation

Importing libraries

In [ ]:
import pandas as pd
import numpy as np
from numpy import array
from keras import layers
from datetime import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Embedding,Dense
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from nltk.tokenize import word_tokenize
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import SGD
from keras.layers import LSTM, SimpleRNN, GRU, Bidirectional, BatchNormalization,Convolution1D,MaxPooling1D, Reshape, GlobalAveragePooling1D
import h5py
from keras import callbacks
from sklearn import metrics

Upload Data

In [ ]:
from google.colab import files
dataset = files.upload()

Saving D1_all_shuffled.csv to D1_all_shuffled.csv


Loading Data

In [ ]:
df = pd.read_csv(r"D1_all_shuffled.csv")
X = df.iloc[:,df.columns!= 'label']
y = df.iloc[:,-1]

print(X)
print(y)

       digitsHostname  PresentAtrateHost  AvgWordLenHost  LongWordLenHost  \
0                   0                  0       10.500000               18   
1                   0                  0        5.000000                9   
2                   5                  0        8.333333               13   
3                   0                  0        5.000000                7   
4                   0                  0        7.666667               15   
...               ...                ...             ...              ...   
83852               0                  0        4.000000                5   
83853               0                  0        5.666667               11   
83854               0                  0        4.333333                7   
83855               2                  0        3.333333                4   
83856               0                  0        5.000000                9   

       HostNameLen  domainLen  digitsDomain  dotsHost  HypensHost  \
0     

In [ ]:
print(X.iloc[0])

digitsHostname        0.000000
PresentAtrateHost     0.000000
AvgWordLenHost       10.500000
LongWordLenHost      18.000000
HostNameLen          22.000000
                       ...    
]                     0.000000
-                     0.000000
_                     0.052632
.                     0.105263
~                     0.000000
Name: 0, Length: 93, dtype: float64


Defining Keras Models

In [ ]:
model = Sequential([
    Convolution1D(64,kernel_size=3,padding="same",activation="relu",input_shape=(93,1)),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),
    Dropout(0.5),

    Convolution1D(128,kernel_size=128,padding="same",activation="relu"),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),
    Dropout(0.5),

    Convolution1D(256,kernel_size=256,padding="same",activation="relu"),
    MaxPooling1D(pool_size=2),
    BatchNormalization(),
    Dropout(0.5),

    Convolution1D(512,kernel_size=512,padding="same",activation="relu"),
    GlobalAveragePooling1D(),
    Dropout(0.5),
    Dense(1,activation='sigmoid')
  ])

print(model.summary())
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 93, 64)            256       
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 46, 64)           0         
 1D)                                                             
                                                                 
 batch_normalization_3 (Batc  (None, 46, 64)           256       
 hNormalization)                                                 
                                                                 
 dropout_4 (Dropout)         (None, 46, 64)            0         
                                                                 
 conv1d_5 (Conv1D)           (None, 46, 128)           1048704   
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 23, 128)         

In [ ]:
for layer in model.layers:
    print(layer.output_shape)

(None, 93, 64)
(None, 46, 64)
(None, 46, 64)
(None, 46, 64)
(None, 46, 128)
(None, 23, 128)
(None, 23, 128)
(None, 23, 128)
(None, 23, 256)
(None, 11, 256)
(None, 11, 256)
(None, 11, 256)
(None, 11, 512)
(None, 512)
(None, 512)
(None, 1)


In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 93, 64)            256       
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 46, 64)           0         
 1D)                                                             
                                                                 
 batch_normalization_3 (Batc  (None, 46, 64)           256       
 hNormalization)                                                 
                                                                 
 dropout_4 (Dropout)         (None, 46, 64)            0         
                                                                 
 conv1d_5 (Conv1D)           (None, 46, 128)           1048704   
                                                                 
 max_pooling1d_4 (MaxPooling  (None, 23, 128)         

In [ ]:
def evaluate_model(X_test,y_test):
    loss, acc = model.evaluate(X_test,y_test)
    return acc

In [ ]:
kf = KFold(n_splits=10)
accuracies = []

start = datetime.now()
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    print(X_train)
    y_train, y_test = y[train_index], y[test_index]
    y_train = y_train.values.reshape(-1,1)
    y_test  = y_test.values.reshape(-1,1)
    print(y_train)
    model.fit(X_train,y_train,batch_size = 64,epochs=10)
    accuracy = evaluate_model(X_test,y_test)
    print(accuracy)
    accuracies.append(accuracy)
end = datetime.now()
td = (end - start).total_seconds()
print("Total Execution Time: ",td)

TRAIN: [ 8386  8387  8388 ... 83854 83855 83856] TEST: [   0    1    2 ... 8383 8384 8385]
       digitsHostname  PresentAtrateHost  AvgWordLenHost  LongWordLenHost  \
8386                0                  0        7.500000               23   
8387                0                  0        4.000000                6   
8388                0                  0        4.000000                7   
8389                0                  0        7.500000               13   
8390                0                  0        4.500000                6   
...               ...                ...             ...              ...   
83852               0                  0        4.000000                5   
83853               0                  0        5.666667               11   
83854               0                  0        4.333333                7   
83855               2                  0        3.333333                4   
83856               0                  0        5.000000      

In [ ]:
print(accuracies)

[0.9285714030265808, 0.9406152963638306, 0.9378726482391357, 0.9393035769462585, 0.9443119764328003, 0.9376341700553894, 0.9470546245574951, 0.9457364082336426, 0.9423971176147461, 0.9507454037666321]


In [ ]:
print(sum(accuracies)/len(accuracies))

0.9414242625236511
